In [1]:
import pandas as pd
import numpy as np
import pickle
from collections import Counter, defaultdict

In [2]:
import sys
import os
def add_sys_path(p):
    p = os.path.abspath(p)
    if p not in sys.path:
        sys.path.append(p)

add_sys_path('..')

In [3]:
import evaluate

In [4]:
import data_split

In [6]:
from evaluate import read_dataset

In [11]:
import json

In [69]:
from tqdm.auto import tqdm, trange

# Чтение и сплит данных

In [13]:
v_ds = read_dataset('../data/training_data/training_verbs.tsv',  lambda x: json.loads(x))

In [16]:
v_ds['АМНИСТИРОВАТЬ']

[['115444-V', '120034-V'], ['117915-V', '135001-V']]

In [17]:
len(v_ds)

6806

In [18]:
6806 * 0.02

136.12

In [20]:
train_share = 0.8
dev_share = 0.02
test1_share = 0.02
test2_share = 0.02
hid_share = 0.14

assert train_share + dev_share + test1_share + test2_share + hid_share == 1

In [21]:
from data_split import hash_float

In [31]:
train, dev, test1, test2, hid = {}, {}, {} , {}, {}

for k, v in v_ds.items():
    h = hash_float(k)
    if h <= train_share:
        train[k] = v
    elif h <= train_share + dev_share:
        dev[k] = v
    elif h <= train_share + dev_share + test1_share:
        test1[k] = v
    elif h <= train_share + dev_share + test1_share + test2_share:
        test2[k] = v
    else:
        hid[k] = v

# Применение бейзлайна

In [132]:
import my_knn
from importlib import reload
reload(my_knn)

<module 'my_knn' from 'C:\\Users\\ddale\\YandexDisk\\code\\NLP\\taxonomy-enrichment\\experiments\\my_knn.py'>

In [33]:
import gensim
# this is araneum 2018 ft model from rusvectores
ft = gensim.models.fasttext.FastTextKeyedVectors.load(
    'C:/Users/ddale/Downloads/NLP/rusvectores/model.model'
)

In [64]:
embedder = my_knn.SentenceEmbedder(ft=ft, n=300)

In [67]:
embedder('привет как дела').shape

(300,)

In [42]:
import xmltodict
with open('../data/ruwordnet/synsets.V.xml', 'r', encoding='utf-8') as f:
    synsets_v_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [50]:
with open('../data/ruwordnet/synset_relations.V.xml', 'r', encoding='utf-8') as f:
    rel_v_raw = xmltodict.parse(f.read(), process_namespaces=True)

In [52]:
verbs_storage = my_knn.SynsetStorage.construct(synsets_v_raw)

7521 49002


In [54]:
rel_df = my_knn.make_rel_df(rel_v_raw, verbs_storage.id2synset)

In [55]:
rel_df.head()

,@parent_id,@child_id,@name,parent,child
0,4223-V,4223-N,POS-synonymy,ЛИВЕНЬ,None
1,4223-V,4223-A,POS-synonymy,ЛИВЕНЬ,None
2,4223-V,5068-N,domain,ЛИВЕНЬ,None
3,129120-V,129121-V,hyponym,"БЫТЬ ИЗВЕСТНЫМ, СЛЫТЬ",СЛАВИТЬСЯ (ПОЛЬЗОВАТЬСЯ ИЗВЕСТНОСТЬЮ)
4,129120-V,148420-V,hypernym,"БЫТЬ ИЗВЕСТНЫМ, СЛЫТЬ",ОТНОШЕНИЯ МЕЖДУ ЛЮДЬМИ


In [63]:
verb_rel_storage = my_knn.RelationStorage() # todo: add forbidden_id
verb_rel_storage.construct_relations(rel_df)


7408
218
5
10317


In [44]:
len(verbs_storage.texts_long)

49002

In [70]:
vecs = np.stack([embedder(t) for t in tqdm(verbs_storage.texts_long) ])

In [71]:
from sklearn.neighbors import KDTree
tree = KDTree(vecs)

In [89]:
hypos = my_knn.hypotheses_knn('ковырять', index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage)
hypos

['111845-V',
 '111846-V',
 '106701-V',
 '111791-V',
 '106772-V',
 '112004-V',
 '115889-V',
 '115208-V',
 '115890-V',
 '127537-V']

In [90]:
for id in hypos:
    print(verbs_storage.id2synset[id]['@ruthes_name'])

ПРОНИКНУТЬ ВНУТРЬ
ПОПАСТЬ (ОКАЗАТЬСЯ, ОЧУТИТЬСЯ)
ДОСТИЧЬ, ДОБИТЬСЯ
КОПАТЬ ЯМУ
УГЛУБИТЬ, УВЕЛИЧИТЬ ГЛУБИНУ (РАССТОЯНИЕ)
РАЗБРОСАТЬ, РАСШВЫРЯТЬ
УЯЗВИТЬ
ОСКОРБИТЬ
ОБИДЕТЬ, ПРИЧИНИТЬ ОБИДУ
ЕХИДНИЧАТЬ, ЯЗВИТЬ


In [121]:
# проверка, что сравнение само с собой всегда получает скор 1
dev_pred = {k: [v0 for val in vals for v0 in val] for k, vals in dev.items()}
mean_ap, mean_rr = evaluate.get_score(dev, dev_pred, k=10)
print(mean_ap, mean_rr)

1.0 1.0


In [119]:
#texts = ttest_test1.one_text.drop_duplicates()
texts = sorted(dev.keys())
small_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(texts)
}


In [122]:
mean_ap, mean_rr = evaluate.get_score(dev, small_hypos, k=10)
print(mean_ap, mean_rr)

0.5837428092042187 0.6332587621178226


# Оценка применения

In [127]:
public_test_verbs = pd.read_csv('../data/public_test/verbs_public.tsv', header=None)
public_test_verbs.columns = ['text']

In [130]:
public_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(public_test_verbs.text)
}

In [133]:
sub = my_knn.dict2submission(public_test_hypos, verbs_storage.id2synset)

In [134]:
sub

,noun,result,result_text
0,АБСОЛЮТИЗИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
1,АБСОЛЮТИЗИРОВАТЬ,7237-V,УПРАВЛЕНЧЕСКАЯ ДЕЯТЕЛЬНОСТЬ
2,АБСОЛЮТИЗИРОВАТЬ,141697-V,ПРЕДСТАВИТЬ В ВИДЕ
3,АБСОЛЮТИЗИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АБСОЛЮТИЗИРОВАТЬ,7936-V,РЕФОРМА
...,...,...,...
1745,ШТОПАТЬ,106495-V,"ИЗГОТОВИТЬ, ВЫРАБОТАТЬ"
1746,ШТОПАТЬ,108856-V,"ОДЕТЬ, ПОКРЫТЬ ОДЕЖДОЙ, ПОКРЫВАЛОМ"
1747,ШТОПАТЬ,118048-V,ПРОШИТЬ ШВОМ
1748,ШТОПАТЬ,107010-V,МЫТЬ ОТ ГРЯЗИ


In [136]:
sub.to_csv('results/verbs_v0_strange.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АБСОЛЮТИЗИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
1,АБСОЛЮТИЗИРОВАТЬ,7237-V,УПРАВЛЕНЧЕСКАЯ ДЕЯТЕЛЬНОСТЬ
2,АБСОЛЮТИЗИРОВАТЬ,141697-V,ПРЕДСТАВИТЬ В ВИДЕ
3,АБСОЛЮТИЗИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
4,АБСОЛЮТИЗИРОВАТЬ,7936-V,РЕФОРМА
5,АБСОЛЮТИЗИРОВАТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
6,АБСОЛЮТИЗИРОВАТЬ,106607-V,"УПРАВЛЯТЬ, РУКОВОДИТЬ"
7,АБСОЛЮТИЗИРОВАТЬ,119125-V,"РАСПРОСТРАНИТЬСЯ, ПОЛУЧИТЬ РАСПРОСТРАНЕНИЕ"
8,АБСОЛЮТИЗИРОВАТЬ,123550-V,"УТОЧНИТЬ, СДЕЛАТЬ ТОЧНЕЕ"
9,АБСОЛЮТИЗИРОВАТЬ,135851-V,ПРИДАТЬ СВОЙСТВО


# Финальная заливка

In [138]:
private_test_verbs = pd.read_csv('../data/private_test/verbs_private.tsv', header=None)
private_test_verbs.columns = ['text']

In [140]:
private_test_hypos = {
    txt: 
    my_knn.hypotheses_knn(
        txt, 
        index=tree, text2vec=embedder, synset_storage=verbs_storage, rel_storage=verb_rel_storage,
        decay=3, 
        k=100, 
        grand_mult=0.5,
    )  
    for txt in tqdm(private_test_verbs.text)
}

In [141]:
sub = my_knn.dict2submission(private_test_hypos, verbs_storage.id2synset)

In [142]:
sub

,noun,result,result_text
0,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
1,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
2,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
3,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
4,АДСОРБИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
...,...,...,...
3495,ЮМОРИТЬ,116284-V,"ВЫСМЕЯТЬ, ОСМЕЯТЬ"
3496,ЮМОРИТЬ,118698-V,"ДЕЙСТВИЕ, ЦЕЛЕНАПРАВЛЕННОЕ ДЕЙСТВИЕ"
3497,ЮМОРИТЬ,116285-V,НАСМЕХАТЬСЯ
3498,ЮМОРИТЬ,115102-V,МУЧИТЬ


In [143]:
sub.to_csv('results/private_verbs_v0_strange.tsv', sep='\t', encoding='utf-8', header=None, index=None)
sub.head(15)

,noun,result,result_text
0,АДСОРБИРОВАТЬ,111097-V,ИЗМЕНЕНИЕ АГРЕГАТНОГО СОСТОЯНИЯ
1,АДСОРБИРОВАТЬ,106629-V,"ИЗМЕНИТЬСЯ, ИЗМЕНЕНИЕ"
2,АДСОРБИРОВАТЬ,106631-V,"ИЗМЕНИТЬ, СДЕЛАТЬ ИНЫМ"
3,АДСОРБИРОВАТЬ,106950-V,"ПРЕВРАТИТЬ (ПРИДАТЬ ИНОЙ ВИД, КАЧЕСТВО)"
4,АДСОРБИРОВАТЬ,106501-V,"ЗАНЯТИЕ, ДЕЯТЕЛЬНОСТЬ"
5,АДСОРБИРОВАТЬ,112139-V,ОЧИСТКА ОТ ПРИМЕСЕЙ
6,АДСОРБИРОВАТЬ,106534-V,"ОБРАБАТЫВАТЬ, ПОДВЕРГАТЬ ОБРАБОТКЕ"
7,АДСОРБИРОВАТЬ,112136-V,ПРОПУСТИТЬ (ДАТЬ ПРОНИКНУТЬ)
8,АДСОРБИРОВАТЬ,107417-V,ОТДЕЛИТЬ ЧАСТЬ ОТ ЦЕЛОГО
9,АДСОРБИРОВАТЬ,145451-V,СЕПАРАЦИЯ СМЕСИ


# А теперь разбираемся

Почему мои локальные скоры в 2-3 раза выше, чем при заливке на сервер?